#목차

1. 간단한 keras 모델 만들기
2. 입력 함수 만들기
3. tf.keras 모델에서 에스티메이터 만들기

In [5]:
import tensorflow_datasets as tfds #TensorFlow Datasets 는 다양한 데이터셋을 TensorFlow에서 활용하기 쉽도록 제공

import tensorflow as tf

import numpy as np

1. 간단한 케라스(Keras) 모델 만들기
케라스에서는 레이어를 모으고 모델을 제작합니다. 모델은 보통 레이어의 그래프로 가장 일반적인 유형의 모델은 바로 레이어를 쌓는 것입니다. tf.keras.Sequential을 이용해 모델을 만듭니다.

단순하고 완전히 연결된 네트워크(즉, 다중 레이어 인식자)를 구축하려면 다음을 수행합니다:


In [15]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)), #4개노드 입력을 받는 입력층, 16개 유닛 가진 연결층 추가
    tf.keras.layers.Dropout(0.2),  #Dropout 함수를 적용할 경우, 과적합을 방지하기 위해 무작위로 특정 노드(입력값)를 0으로 만든다.
                                    # 인풋 데이터에 20%의 노드들을 무작위로 0으로 만드는 드롭아웃을 적용.
    tf.keras.layers.Dense(3)       # 이렇게 20% 정도가 무시된 값이 3번째 레이어에 입력되어 충 3개의 값을 출력
])

모델을 컴파일하고 모델 구성을 확인하기 위한 모델 요약을 확인합니다.

In [7]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer='adam') #모델의 학습 중에 역전파를 통한 가중치 최적화를 위한 기울기 방향에 대한 경사하강을 위한 방법으로 adam
                                #손실함수로 다중 분류의 Cross Entropy Error인 
                                #‘sparse_categorical_crossentropy’를 지정하였습니다.
                                #그리고 모델 평가를 위한 평가 지표로 ‘accuracy’를 지정하였습니다. 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                80        
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 131
Trainable params: 131
Non-trainable params: 0
_________________________________________________________________


2. 입력 함수 만들기

Datasets API를 사용하여 대규모 데이터셋이나 다중 장치(multi-device) 학습으로 확장할 수 있습니다.

에스티메이터(Estimator)는 입력 파이프라인을 만드는 시기와 방법을 제어해야 합니다. 이를 위해 직접 생성한 입력 함수 또는 input_fn이 필요합니다. Estimator는 인수 없이 이 함수를 호출합니다. input_fn은 tf.data.Dataset을 반환해야 합니다.

What is Estimators?Permalink
추상화(abstraction) 이란 코드를 특정한 목적으로 일반화하여 기존 코드 ‘위에 올라가는’ 코드의 계층을 의미한다. 관련있는 High-Level 기능을 묶는 방식의 재구성을 통해 코드를 감싸서 추상화한다. 따라서 쉽게 코딩할 수 있고, 가독성이 좋으며, 코드가 간소화된다.
이러한 추상화 라이브러리를 살펴보면 다음과 같은 종류들이 존재한다.

tf.estimator또한 Keras와 같이 관련있는 High-Level 기능을 묶는 방식의 재구성을 통해 Code를 추상화하는 방법이다.

tf.estimator는 다음과 같은 기능들을 포함하고 있습니다. / training,evaluation,prediction,export for serving

In [8]:
def input_fn():
  split = tfds.Split.TRAIN  # train dataset을 손쉽게 로드, test dataset도 마찬가지로 split=tfds.Split.TEST 만 지정해 주면 로드 됩니다.
    # 모델의 학습시키기 위해 데이터셋을 train과 validation 으로 나누는데, 이것을 데이터셋 라이브러리인 tfds의 Split함수로 나눌수 있습니다. 

그 후에 iris 데이터셋을 불러오고 
  dataset = tfds.load('iris', split=split, as_supervised=True)  #Iris Dataset을 다운받고 Dict Type으로서 설정하는 과정을 거친다.
     # 
  dataset = dataset.map(lambda features, labels: ({'dense_input':features}, labels)) 
    # map 함수를 이용해서 dataset의 각 멤버에 사용자 지정 함수를 적용
    # iris데이터셋의 features와 labels을 매핑해주는 것입니다. label은 정답이라고 생각하시면 됩니다. . 
    # as_supervised = True 라고 이미 했기 때문에, 정답을 알려줘야 하기 때문입니다. 
as_supervised = True 라고 이미 했기 때문에, 정답을 알려줘야 하기 때문입니다. 

as_supervised = True 라고 이미 했기 때문에, 정답을 알려줘야 하기 때문입니다. 
  dataset = dataset.batch(32).repeat()  # batch란, 한 번에 계산할 데이터의 양
  return dataset

input_fn을 테스트해봅니다.

In [14]:
for features_batch, labels_batch in input_fn().take(1):
  print(features_batch)
  print(labels_batch)

    'features': Tensor(shape=(4,), dtype=tf.float32),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'features': Tensor(shape=(4,), dtype=tf.float32),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'features': Tensor(shape=(4,), dtype=tf.float32),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


{'dense_input': <tf.Tensor: id=2452, shape=(32, 4), dtype=float32, numpy=
array([[5.1, 3.4, 1.5, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [5.7, 2.8, 4.5, 1.3],
       [6.8, 3.2, 5.9, 2.3],
       [5.2, 3.4, 1.4, 0.2],
       [5.6, 2.9, 3.6, 1.3],
       [5.5, 2.6, 4.4, 1.2],
       [5.5, 2.4, 3.7, 1. ],
       [4.6, 3.4, 1.4, 0.3],
       [7.7, 2.8, 6.7, 2. ],
       [7. , 3.2, 4.7, 1.4],
       [4.6, 3.2, 1.4, 0.2],
       [6.5, 3. , 5.2, 2. ],
       [5.5, 4.2, 1.4, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5. , 3.5, 1.3, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [4.8, 3. , 1.4, 0.1],
       [6.5, 3. , 5.8, 2.2],
       [7.6, 3. , 6.6, 2.1],
       [6.7, 3.3, 5.7, 2.1],
       [7.9, 3.8, 6.4, 2. ],
       [6.7, 3. , 5.2, 2.3],
       [5.8, 4. , 1.2, 0.2],
       [6.3, 2.5, 5. , 1.9],
       [5. , 3. , 1.6, 0.2],
       [6.9, 3.1, 5.1, 2.3],
       [6.1, 3. , 4.6, 1.4],
       [5.8, 2.7, 4.1, 1. ],
       [5.2, 2.7, 3.9, 1.4],
       [6.7, 3. , 5. , 1.7],
       [5.7, 2.6, 3.5, 1. ]

3. tf.keras 모델에서 에스티메이터(Estimator) 만들기

tf.keras.estimator.model_to_estimator를 이용해 모델을 tf.estimator.Estimator로 변환함으로써 tf.keras.Model을 tf.estimator API로 학습시킬 수 있습니다.

In [11]:
import tempfile
model_dir = tempfile.mkdtemp()  # 이제 모델이 생성되고 실행하면서 발생하는 데이터들을 로깅과 모니터링 하기 위한 임시 폴더 생성합니다.
# moder_dir 는 원본 코드에서는 없는 파라미터,  model_dir 파라미터에 output이 생길 폴더의 경로를 지정해주어야한다.
#만약 소스코드가 C:\Users\SJHong\Documents에 있다고 하자. 그리고 output 폴더도 Documents 폴더에 지정하기를 원한다면, 
# model_dir = "../Output" 과 같이 상대경로로 간단하게 지정할 수 있다.

keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)  
# Keras Model을 tf.keras.estimator.model_to_estimator를 통하여 tf.estimator로서 변경하여 결과를 확인한다.
# 위에서 Keras -> Estimator로서 변경하였으므로
# Keras Model을 사용할 수 있는 Folder와 Data 전처리 + Keras Model 인 Estimator를 사용할 수 있다.

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using the Keras model provided.


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\김희준\\AppData\\Local\\Temp\\tmp07ynpaif', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025290768308>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\김희준\\AppData\\Local\\Temp\\tmp07ynpaif', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025290768308>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


에스티메이터(Estimator)를 학습시키고 평가합니다.

In [12]:
keras_estimator.train(input_fn=input_fn, steps=500) # steps는 함수가 최대 몇 번 실행될지를 의미한다. 총 500번 학습
eval_result = keras_estimator.evaluate(input_fn=input_fn, steps=10) # 평가는 10번
print('평가 결과: {}'.format(eval_result))

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


    'features': Tensor(shape=(4,), dtype=tf.float32),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'features': Tensor(shape=(4,), dtype=tf.float32),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


    'features': Tensor(shape=(4,), dtype=tf.float32),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
})> could not be transformed and will be executed as-is. Please report this to the AutoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: Bad argument number for Name: 3, expecting 4


...

INFO:tensorflow:Saving dict for global step 500: global_step = 500, loss = 0.48727512


INFO:tensorflow:Saving dict for global step 500: global_step = 500, loss = 0.48727512


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: C:\Users\김희준\AppData\Local\Temp\tmp07ynpaif\model.ckpt-500


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 500: C:\Users\김희준\AppData\Local\Temp\tmp07ynpaif\model.ckpt-500


평가 결과: {'loss': 0.48727512, 'global_step': 500}
